In [10]:
!pip3 install pandas

You should consider upgrading via the 'd:\diploma\kotlin-gan\venv\scripts\python.exe -m pip install --upgrade pip' command.


In [11]:
from sympy import Derivative, symbols, exp, hessian
import numpy as np

import tracemalloc
import time
import math

from BaseOptimizer import BaseOptimizer
from ConjGradient import ConjGradient
from Newton import Newton
from FunctionWrapper import FunctionWrapper

In [6]:
x, y = symbols('x y')
fs = [
    100 * (y - x) ** 2 + (1 - x) ** 2,
    100 * (y - x ** 2) ** 2 + (1 - x) ** 2,
    2 * exp(-((x - 1) / 2) ** 2 - (y - 1) ** 2) + 3 * exp(-((x - 2) / 3) ** 2 - ((y - 3) / 2) ** 2)
]
f_grads = [(Derivative(f, x).doit(), Derivative(f, y).doit()) for f in fs]
f_hessians = [hessian(f, [x, y]) for f in fs]

In [8]:
type(x)

sympy.core.symbol.Symbol

In [16]:
starting_points = [
    np.array([0, 0]),
    np.array([5, 0]),
    np.array([-4, -1]),
    np.array([0.1, 2.4]),
    np.array([0.01, 0.005]),
]

algs = [Newton, ConjGradient]
opts = []
times = []
memorys = []
mns = []
for (f, grad, hessian) in zip(fs, f_grads, f_hessians):
    f = FunctionWrapper(f)
    _opts = {alg: [] for alg in algs}
    _mns = { alg: [] for alg in algs }
    _times = []
    _mems = []
    for alg in algs:
        # Logs
        for ip in starting_points:
            try:
                opt = alg(f, 1e-6)
                mn = opt.optimize(ip)
                _mns[alg].append(mn)
                _opts[alg].append(opt)
                print(mn, opt.get_iterations_cnt())
            except RuntimeError as e:
                if hasattr(e, 'message'):
                    print(e.message)
                else:
                    print(e)

        # Time/Memory
        tracemalloc.start()
        mn_time = 1e9
        for i in range(5):
            st = np.random.randn(2) * 1.5
            opt = alg(f, 1e-6, preserve_logs=False)
            try:
                start = datetime.now()
                opt.optimize(st)
                elapsed_time = datetime.now() - start
                elapsed_micros = elapsed_time.seconds * 1e6 + elapsed_time.microseconds
                mn_time = min(mn_time, elapsed_micros)
            except RuntimeError as e:
                print(e)
                pass
        peek_mem = tracemalloc.get_traced_memory()[1] / 1024
        tracemalloc.stop()

        _times.append(mn_time / 1e3)
        _mems.append(peek_mem)
    opts.append(_opts)
    times.append(_times)
    mems.append(_mems)
    mns.append(_mns)

TypeError: 'Add' object is not callable